### DeepSeek Inference

In [1]:
#!pip install flash_attn vllm

In [2]:
# Основные библиотеки
import os
import io
import json
import ast
import re
import time
import tempfile
import subprocess
import requests
import random

import pandas as pd
import torch
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

from typing import Optional
from collections import Counter
from multiprocessing import Process, Queue
from tqdm import tqdm

from cleantext import clean
from better_profanity import profanity

2024-10-19 12:24:55.819596: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 12:24:55.906803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-19 12:24:58.018615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-19 12:25:00,562	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
solutions = pd.read_excel('../data/for_teams/test/solutions.xlsx')
tasks = pd.read_excel('../data/for_teams/test/tasks.xlsx')
tests = pd.read_excel('../data/for_teams/test/tests.xlsx')
tests_with_errors = pd.read_csv('../data/new_data/test/test_solutions_with_flake8.csv')

In [4]:
solutions = tests_with_errors.merge(tasks[['id', 'description', 'author_solution']], left_on='task_id', right_on='id', how='left', suffixes=('', '_task'))

In [5]:
solutions

,Unnamed: 0,id,task_id,student_solution,author_comment,author_comment_embedding,message,line_number,line,id_task,description,author_solution
0,0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,SyntaxError: unterminated f-string literal (de...,6.0,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,SyntaxError: unterminated string literal (dete...,6.0,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
...,...,...,...,...,...,...,...,...,...,...,...,...
320,320,725,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
321,321,726,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
322,322,727,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
323,323,728,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...


In [6]:
max_model_len, tp_size = 8192, 1
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model=model_name, tensor_parallel_size=tp_size, max_model_len=max_model_len, trust_remote_code=True, enforce_eager=True)
sampling_params = SamplingParams(temperature=0.0, max_tokens=256, stop_token_ids=[tokenizer.eos_token_id])

INFO 10-19 12:25:05 config.py:107] Replacing legacy 'type' key with 'rope_type'
WARNING 10-19 12:25:18 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-19 12:25:18 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='deepseek-ai/DeepSeek-Coder-V2-Lite-Base', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Coder-V2-Lite-Base', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), o

OutOfMemoryError: CUDA out of memory. Tried to allocate 704.00 MiB. GPU 0 has a total capacity of 5.77 GiB of which 341.75 MiB is free. Including non-PyTorch memory, this process has 5.05 GiB memory in use. Of the allocated memory 4.95 GiB is allocated by PyTorch, and 14.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Функция для генерации комментария
def generate_comment(task_description, student_solution, synt_errors):
    system_message = '''
    Ты - профессиональный программист и учитель.
    Я пришлю тебе условие задачи, верное решение и мое решение с ошибками.
    Дай крайне короткий ответ в одном предложении об ошибке в решении.
    Ошибки могут быть синтаксические и логические.
    Объясни ошибку через смысл задания. Ни в коем случае не ссылайся на реализацию в верном решении.
    Ни в коем случае не присылай исправленный код решения.
    Используй следующие фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде.
    '''

    user_message = f'''
    Условие задачи:
    {task_description}

    Мое решение с ошибкой:
    {student_solution}

    Синтаксические ошибки в решении:
    {synt_errors}
    '''

    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

    prompt_token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

    outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

    result = outputs[0].outputs[0].text
    return result

In [ ]:
# Пример задачи
task_description = "Реализуйте программу, которая напечатает стоимость товара после применения скидки."
student_solution = "discount = float(input())\nmoney = int(input())\nprint(money * discount"
synt_errors = "SyntaxError: unterminated string literal"

author_comment = generate_comment(task_description, student_solution, synt_errors)

print(f"Сгенерированный комментарий: {author_comment}")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.07s/it, est. speed input: 121.15 toks/s, output: 13.03 toks/s]

Сгенерированный комментарий:  В вашем коде отсутствует закрывающая кавычка для строки, которая выводится на экран.


In [ ]:
file_name = '../jailbreak/obscene_lexicon.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    bad_words = file.read().split('\n')

def clean_answer(text):
    def remove_code_from_text(text):
        code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
        fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
        cleaned_text = code_pattern.sub('', text)
        cleaned_text = fenced_code_pattern.sub('', cleaned_text)
        return cleaned_text

    def remove_obscene_lexicon(text):
        cleaned_text = clean(text, extra_spaces=True, stemming=False, stopwords=True, lowercase=False, stp_lang='english' )
        cleaned_text = profanity.censor(cleaned_text)
        for word in bad_words:
            bad_words_re = re.compile(r'\b(' + word + r')\b', re.IGNORECASE)
            cleaned_text = bad_words_re.sub(lambda match: '***', cleaned_text)
        return cleaned_text

    text = remove_code_from_text(text)
    text = remove_obscene_lexicon(text)
    return text

In [ ]:
author_comments = []

for idx, row in solutions.iterrows():
    task_description = row['description']
    student_solution = row['student_solution']
    synt_errors = row['message'] if not pd.isna(row['message']) else "нет синтаксических ошибок"

    author_comment = generate_comment(task_description, student_solution, synt_errors)
    author_comment = clean_answer(author_comment)
    author_comments.append({"id": row['id'], "author_comment": author_comment})

submit_df = pd.DataFrame(author_comments)

submit_df[['id', 'author_comment']].head()

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.72s/it, est. speed input: 235.41 toks/s, output: 16.72 toks/s]


,id,author_comment
0,0,"Вы некорректно выполняете условия задания, ис..."
1,1,В вашем решении не закрытый кавычки в f-строк...
2,2,В вашем коде отсутствует закрывающая кавычка ...
3,3,"Выполняет условия задания, но некорректно вып..."
4,4,Вы некорректно выполняет условия задания. Вы ...


In [ ]:
# Загрузка модели для эмбеддингов

model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = tokenizer(
        text=sentence,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding

def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])

embeddings = []

for comment in tqdm(submit_df['author_comment'], desc="Генерация эмбеддингов"):
    embedding = get_sentence_embedding(comment)
    embedding_str = embedding2string(embedding)
    embeddings.append(embedding_str)

submit_df['author_comment_embedding'] = embeddings

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Генерация эмбеддингов: 100%|██████████| 325/325 [00:19<00:00, 16.67it/s]


In [ ]:
submit_df

,id,author_comment,author_comment_embedding
0,0,"Вы некорректно выполняете условия задания, ис...",-0.5607446432113647 -1.5837291479110718 0.0392...
1,1,В вашем решении не закрытый кавычки в f-строк...,0.01653958112001419 -1.7250393629074097 -0.218...
2,2,В вашем коде отсутствует закрывающая кавычка ...,-0.5205071568489075 -1.3678265810012817 0.2518...
3,3,"Выполняет условия задания, но некорректно вып...",-0.4490863084793091 -1.5153733491897583 0.1720...
4,4,Вы некорректно выполняет условия задания. Вы ...,-0.3266308009624481 -1.8770661354064941 -0.155...
...,...,...,...
320,725,В вашем коде условия задания некорректно выпо...,-0.4523405432701111 -1.5485786199569702 0.0891...
321,726,"Ошибка в вашем коде заключается в том, что фу...",-0.25639352202415466 -1.2132973670959473 0.560...
322,727,"Выполняет условия задания, но некорректно вып...",-0.45749685168266296 -1.4911483526229858 -0.19...
323,728,"Выполняет условия задания, но некорректно вып...",-0.47172752022743225 -1.4901357889175415 0.022...


In [ ]:
# Переименование столбца 'id' на 'solution_id' в submit_df
submit_df.rename(columns={'id': 'solution_id'}, inplace=True)
submit_df.to_csv('../data/deepseek/submit_deepseek.csv', index=False)